In [1]:
import pickle
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from src.core import LoadBlob, SynthLeader, DataProcessor

In [3]:
# if not os.path.exists("./data/bmk2018.pkl"):
#     data = blob.load_csv_from_azure_storage(blob_name = 'BMK_2018.csv')
#     # data['RaterType'] = data['RaterType'].str.lower()

#     if not os.path.exists("./data"):
#         os.mkdir("./data")
#     with open('./data/bmk2018.pkl', 'wb') as pkl:
#         pickle.dump(data, pkl, protocol=pickle.HIGHEST_PROTOCOL)
# else:
#     try:
#         with open('./data/bmk2018.pkl', 'rb') as pkl:
#             data = pickle.load(pkl)
#     except EOFError:
#         print("Error: End of file reached unexpectedly. Check for file corruption or empty file.")
#     except FileNotFoundError:
#         print("Error: File not found. Verify the file path.")
#     except Exception as e:
#         print(f"An unexpected error occurred: {e}")

In [5]:
data_processor = DataProcessor(data="BMK_2018.csv")

ValueError: Invalid URL: https://

In [5]:
from dotenv import load_dotenv
load_dotenv()

os.environ["AZURE_ACCOUNT_URL"]

KeyError: 'AZURE_ACCOUNT_URL'

In [ ]:
rater_count = count_raters(data)
rater_count

In [ ]:
select_data = data[["ESI_Key", "RaterType"]]
peer_counts = select_data[select_data["RaterType"]
                          == "Peer"].groupby("ESI_Key").size()
boss_counts = select_data[select_data["RaterType"]
                          == "Boss"].groupby("ESI_Key").size()
direct_report_counts = select_data[select_data["RaterType"]
                                   == "Direct Report"].groupby("ESI_Key").size()

# Plotting
plt.figure(figsize=(14, 6))

# Histogram for peer counts
plt.subplot(1, 3, 1)  # 1 row, 3 cols, 1st subplot
plt.hist(peer_counts, bins=10, color='blue', alpha=0.7)
plt.title('Distribution of Peer Rater Counts')
plt.xlabel('Counts')
plt.ylabel('Frequency')


# Histogram for boss counts
plt.subplot(1, 3, 2)  # 1 row, 3 cols, 2nd subplot
plt.hist(boss_counts, bins=10, color='green', alpha=0.7)
plt.title('Distribution of Boss Rater Counts')
plt.xlabel('Counts')
plt.ylabel('Frequency')

# Histogram for direct report counts
plt.subplot(1, 3, 3)  # 1 row, 3 cols, 3rd subplot
plt.hist(direct_report_counts, bins=10, color='red', alpha=0.7)
plt.title('Distribution of Direct Report Rater Counts')
plt.xlabel('Counts')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
print("Peer: ", peer_counts.min(), peer_counts.max())
print("Boss: ", boss_counts.min(), boss_counts.max())
print("Direct Report: ", direct_report_counts.min(), direct_report_counts.max())

In [ ]:
# Group by 'ESI_Key' and count unique 'RaterType' values
unique_rater_counts = data.groupby("ESI_Key")["RaterType"].nunique()

# Get the `ESI_Key` of self raters with more than 4 raters
esi_full_raters = unique_rater_counts[unique_rater_counts >= 4]

data = data[data["ESI_Key"].isin(esi_full_raters.index)]

In [ ]:
data["RaterType"].unique()

# there are Superior and Other raters that is not important for the work. Removing it
data = data[~data["RaterType"].isin(['Superior', 'Other'])]

In [ ]:
demo_cols = [
    "Age",
    "Race",
    "OrganizationSector",
    "OrganizationType",
    "RaterType",
    "OrganizationLevel",
    "Gender"
]

In [ ]:
# all the items cols
items_cols = ["ESI_Key", "BMK_S01_Strategic", "BMK_S02_Quick", "BMK_S03_Decisive", "BMK_S04_Change", "BMK_S05_Leading",
              "BMK_S06_Confront", "BMK_S07_Participative", "BMK_S08_Build", "BMK_S09_Compassion", "BMK_S10_Putting", "BMK_S11_Respect"]

In [ ]:
# real_data[(real_data["ESI_Key"] == 4421981) & (real_data["RaterType"] == "Direct Report")]
# demo_cols + items_cols

In [ ]:
real_data = data.loc[:, demo_cols + items_cols]

In [ ]:
df_self = real_data[real_data["RaterType"] == "Self"].drop_duplicates(
    subset=["ESI_Key"], keep="last")
df_self = df_self[items_cols]
df_self

In [ ]:
df_others = real_data[real_data["RaterType"] != "Self"]

In [ ]:
df_others[["ESI_Key", "RaterType"]].groupby(
    ["ESI_Key", "RaterType"]).value_counts()

In [ ]:

# After unstacking, the columns will be multi-level, where the top level is the original columns
# and the second level is the 'RaterType'. You might want to flatten this for easier usage.
# Flatten the column names if necessary. Here's one way to do it:

In [ ]:
df_others = df_others.groupby(["ESI_Key", "RaterType"])[items_cols[1:]].mean()
# df_others[["ESI_Key","RaterType"]].groupby(["ESI_Key", "RaterType"]).value_counts()
df_others = df_others.unstack(level='RaterType')
df_others.columns = ['{}_{}'.format(col[0], col[1])
                     for col in df_others.columns]
# df_others

In [ ]:
real_data = pd.merge(df_self, df_others, on="ESI_Key", how="left")
# real_data

In [ ]:
real_data.columns = [c.replace(' ', '_') for c in real_data.columns]
real_data.columns

In [ ]:
# real_data["ESI_Key"] = real_data["ESI_Key"].astype('category')
# real_data #.groupby(["ESI_Key"]).count()

### Only use selected columns

This will consider FormActualKey as primary key

In [ ]:
# find numeric columns

numeric_cols_df = real_data.select_dtypes(include=[np.number])

In [ ]:
correlation_matrix = numeric_cols_df[2:].corr()
correlation_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:

# real_data = data[[
#     "ESI_Key",
#     "RaterType",
#     "BMK_S01_Strategic",
#     "BMK_S02_Quick",
#     "BMK_S03_Decisive",
#     "BMK_S04_Change",
#     "BMK_S05_Leading",
#     "BMK_S06_Confront",
#     "BMK_S07_Participative",
#     "BMK_S08_Build",
#     "BMK_S09_Compassion",
#     "BMK_S10_Putting",
#     "BMK_S11_Respect"]]
# real_data

In [ ]:
real_data.columns

#### Identify distribution of columns

In [ ]:
# from src.distribution import test_col_distributions

# results = test_col_distributions(data=real_data.iloc[:,3:5])
# results

#### Create metadata

In [ ]:
# df_self.dtypes

In [ ]:
from sdv.metadata import SingleTableMetadata
metadata = SingleTableMetadata()

metadata.detect_from_dataframe(real_data)
data_dict = metadata.to_dict()

metadata.validate()

In [ ]:
for c in real_data.columns[1:]:
    metadata.update_column(
        column_name=c,
        sdtype='numerical',
        computer_representation='Float')

In [ ]:
# metadata.to_dict()

In [ ]:
# # There are some column types that metadata discovered.
# incorrect_type_cols = real_data.columns[7:]
# incorrect_type_cols

### 1. Train synthesizer
##### Using Gaussian-Copula Synthesizer

In [ ]:
from sdv.single_table import GaussianCopulaSynthesizer

In [ ]:
# step 1: create the synthesizer
synthesizer = GaussianCopulaSynthesizer(
    metadata,
    enforce_min_max_values=True,
    # enforcing_rounding=False,
    numerical_distributions={

    },
    default_distribution='norm'
)

# step 2: Train the synthesizer
synthesizer.fit(real_data)

In [ ]:
# step 3: Generate synthetic data
synthetic_data = synthesizer.sample(num_rows=100)

In [ ]:
real_data[0:100]

In [ ]:
synth_numeric_df = synthetic_data.select_dtypes(include=[np.number])
synth_corr = synth_numeric_df.corr()

In [ ]:
synth_corr.style.background_gradient(cmap='coolwarm')

### 2. Train Synthesizer
#### Using CTGAN

In [ ]:
from sdv.single_table import CTGANSynthesizer

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
numeric_cols = real_data.select_dtypes(include=[np.number]).columns

In [ ]:
# normalize the data
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()

# real_data_norm = real_data.copy()
# real_data_norm[numeric_cols] = scaler.fit_transform(real_data[numeric_cols]).copy()

In [ ]:
from sdv.metadata import SingleTableMetadata
metadata = SingleTableMetadata()

metadata.detect_from_dataframe(real_data)
data_dict = metadata.to_dict()

metadata.validate()

In [ ]:
# metadata.to_dict()

In [ ]:
real_data

In [ ]:
model_name = "./data/bmk2018_ctgan_comp.pkl"
if os.path.exists(model_name):
    synthesizer = CTGANSynthesizer.load(filepath=model_name)

else:
    synthesizer = CTGANSynthesizer(metadata,
                                   epochs=500,
                                   enforce_rounding=True,
                                   enforce_min_max_values=True,
                                   verbose=True,
                                   cuda=True)
    synthesizer.fit(real_data)

    synthesizer.save(filepath=model_name)

In [ ]:
synthesizer.get_loss_values()

In [ ]:
ctgan_data = synthesizer.sample(num_rows=15000)

In [ ]:
ctgan_data

In [ ]:
ctgan_data[2:]

In [ ]:
correlation_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
ctgan_corr = ctgan_data.corr()
ctgan_corr.style.background_gradient(cmap='coolwarm')

In [ ]:
corr_diff = ctgan_corr - correlation_matrix
corr_diff.style.background_gradient(cmap="coolwarm")

In [ ]:
from sdv.evaluation.single_table import run_diagnostic

diagnostic = run_diagnostic(
    real_data=real_data,
    synthetic_data=ctgan_data,
    metadata=metadata
)

In [ ]:
diagnostic.get_details(property_name='Data Validity')

In [ ]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data,
    ctgan_data,
    metadata
)

In [ ]:
quality_report.get_details('Column Shapes')
quality_report.get_details('Data Validity')

#### Correlation

In [ ]:
from sdmetrics.column_pairs import CorrelationSimilarity

CorrelationSimilarity.compute(
    real_data=real_data[['BMK_S15_SelfAware', 'BMK_D03_DiffChange']],
    synthetic_data=ctgan_data[['BMK_S15_SelfAware', 'BMK_D03_DiffChange']],
    coefficient='Pearson'
)

In [ ]:
from sdmetrics.column_pairs import ContingencySimilarity

ContingencySimilarity.compute(
    real_data=real_data[["ESI_Key", "FormActualKey", "BMK_S01_Strategic", "BMK_S02_Quick", "BMK_S03_Decisive", "BMK_S04_Change", "BMK_S05_Leading", "BMK_S06_Confront", "BMK_S07_Participative", "BMK_S08_Build", "BMK_S09_Compassion",
                         "BMK_S10_Putting", "BMK_S11_Respect", "BMK_S12_Taking", "BMK_S13_Composure", "BMK_S14_Balance", "BMK_S15_SelfAware", "BMK_S16_Career", "BMK_D01_Interper", "BMK_D02_DiffBuild", "BMK_D03_DiffChange", "BMK_D04_Failure", "BMK_D05_Narrow"]],
    synthetic_data=ctgan_data[["ESI_Key", "FormActualKey", "BMK_S01_Strategic", "BMK_S02_Quick", "BMK_S03_Decisive", "BMK_S04_Change", "BMK_S05_Leading", "BMK_S06_Confront", "BMK_S07_Participative", "BMK_S08_Build", "BMK_S09_Compassion",
                               "BMK_S10_Putting", "BMK_S11_Respect", "BMK_S12_Taking", "BMK_S13_Composure", "BMK_S14_Balance", "BMK_S15_SelfAware", "BMK_S16_Career", "BMK_D01_Interper", "BMK_D02_DiffBuild", "BMK_D03_DiffChange", "BMK_D04_Failure", "BMK_D05_Narrow"]]
)

In [ ]:
real_data[['BMK_S02_Quick']].corrwith(ctgan_data[['BMK_S02_Quick']])

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Initialize an empty correlation matrix
correlation_matrix = pd.DataFrame()

# Iterate through columns in df1 and calculate correlation with columns in df2
for column in real_data.columns:
    correlations = real_data[column].corr(ctgan_data)
    correlation_matrix[column] = correlations

# Create a heatmap for visualization
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)

# Add labels and title
plt.title('Correlation Between Columns in df1 and Columns in df2')
plt.xlabel('df2 Columns')
plt.ylabel('df1 Columns')

# Show the plot
plt.show()

#### Visualization

In [ ]:
from sdv.evaluation.single_table import get_column_plot

fig = get_column_plot(
    real_data=data,
    synthetic_data=ctgan_data,
    metadata=metadata,
    plot_type='distplot',
    column_name='BMK_D02_DiffBuild'
)

fig.show()

In [ ]:
from sdv.evaluation.single_table import get_column_plot

fig = get_column_plot(
    real_data=data,
    synthetic_data=ctgan_data,
    metadata=metadata,
    plot_type='bar',
    column_name='BMK_D02_DiffBuild'
)

fig.show()